# **PRACTICA 2**

In [61]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import style
style.use('ggplot') or plt.style.use('ggplot')

# Preprocesado y modelado
# ==============================================================================
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

Lo primero que vamos a hacer es cargar los datos:

In [67]:
df = pd.read_csv('stars_data.csv')

Observamos que hay datos que pueden identificarse de varias formas, entonces debemos agrupar dichos dichas variables categoricas, hacemos el mapping:

In [51]:
# Obtenemos toda la serie de colores que existe en el dataset
df.Color.unique()

array(['Red', 'Blue White', 'White', 'Yellowish White', 'Blue white',
       'Pale yellow orange', 'Blue', 'Blue-white', 'Whitish',
       'yellow-white', 'Orange', 'White-Yellow', 'white', 'yellowish',
       'Yellowish', 'Orange-Red', 'Blue-White'], dtype=object)

In [54]:
# Establecemos una unificación de colores:
mapping_colors = {
    'Red': ['Red'],
    'Orange-Red': ['Orange-Red'],
    'Orange': ['Orange'],
    'Yellow-Orange': ['Pale yellow orange'],
    'Yellow': ['yellowish', 'Yellowish'],
    'White-Yellow': ['Yellowish White', 'yellow-white', 'White-Yellow'],
    'White': ['White', 'Whitish', 'white'],
    'Blue-White': ['Blue White', 'Blue white', 'Blue-white', 'Blue-White'],
    'Blue': ['Blue'],
}

In [57]:
# Mapeamos los datos
for target_color, source_colors in mapping_colors.items():
  df.loc[df.Color.isin(source_colors), 'Color'] = target_color
df

,Temperature,L,R,A_M,Color,Spectral_Class
0,3068,0.002400,0.1700,16.12,Red,M
1,3042,0.000500,0.1542,16.60,Red,M
2,2600,0.000300,0.1020,18.70,Red,M
3,2800,0.000200,0.1600,16.65,Red,M
4,1939,0.000138,0.1030,20.06,Red,M
...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,Blue,O
236,30839,834042.000000,1194.0000,-10.63,Blue,O
237,8829,537493.000000,1423.0000,-10.73,White,A
238,9235,404940.000000,1112.0000,-11.23,White,A


Comprobamos que los datos están unificados:

In [58]:
# Comprobamos que los valores están unificados.
df.Color.unique()

array(['Red', 'Blue-White', 'White', 'White-Yellow', 'Yellow-Orange',
       'Blue', 'Orange', 'Yellow', 'Orange-Red'], dtype=object)

In [63]:
# Obtenemos los colores ordenados (directamente del mapping!)
colors = list(mapping_colors.keys())
colors

['Red',
 'Orange-Red',
 'Orange',
 'Yellow-Orange',
 'Yellow',
 'White-Yellow',
 'White',
 'Blue-White',
 'Blue']

# **1. ENCODING**

Defino las 'Spectral Class' que existen, o sea establezco sus valores:

In [59]:
spectral = ['O','B','A','F','G','K','M']

### 1.1 OneHot

### 1.2 Ordinal

In [71]:
from sklearn.preprocessing import OrdinalEncoder

ordinal = OrdinalEncoder(categories=[colors, spectral])

ct = ColumnTransformer(['OrdinalEncoder', ordinal, ['Color', 'Spectral_Class']], remainder='passthrough')

pip

pipe.fit_transform(df)[0]

TypeError: 'OrdinalEncoder' object is not iterable

# **2. SCALING**

### 2.1 Con escalador

### 2.2 Sin escalador

# **3. REDUCCIÓN DE DIMENSIONES**

### 3.1 None

### 3.2 PCA

# **4. CLUSTERING**

### 4.1 KMeans

### 4.2 Hierarchical/DBScan

In [46]:
# Ver las primeras filas y obtener información básica
print(df.head())
print(df.info())

# Preprocesamiento: Escalar las características numéricas
features = ['Temperature', 'L', 'R', 'A_M']
df[features] = scale(df[features])

# Encoding para datos categóricos si es necesario
df = pd.get_dummies(df, columns=['Spectral_Class', 'Color'])

# Revisar los datos transformados
print(df.head())

   Temperature         L         R       A_M  Spectral_Class_A  \
0    -0.779382 -0.598624 -0.459210  1.116745             False   
1    -0.782110 -0.598624 -0.459241  1.162414             False   
2    -0.828477 -0.598624 -0.459342  1.362213             False   
3    -0.807496 -0.598624 -0.459229  1.167171             False   
4    -0.897819 -0.598624 -0.459340  1.491607             False   

   Spectral_Class_B  Spectral_Class_F  Spectral_Class_G  Spectral_Class_K  \
0             False             False             False             False   
1             False             False             False             False   
2             False             False             False             False   
3             False             False             False             False   
4             False             False             False             False   

   Spectral_Class_M  ...  Color_Pale yellow orange  Color_Red  Color_White  \
0              True  ...                     False       True 

KeyError: "None of [Index(['Spectral_Class', 'Color'], dtype='object')] are in the [columns]"

In [42]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_data(df, x_col, y_col):
    """
    Función para crear un gráfico de dispersión de dos columnas en un DataFrame.

    Parámetros:
    - df (pandas.DataFrame): El DataFrame que contiene los datos.
    - x_col (str): Nombre de la columna que se usará para el eje X.
    - y_col (str): Nombre de la columna que se usará para el eje Y.
    """

    # Creación de la figura y el eje
    fig, ax = plt.subplots(figsize=(6, 3.84))
    
    # Gráfico de dispersión
    ax.scatter(
        x = df[x_col],
        y = df[y_col],
        c = 'white',       # Color de los puntos
        edgecolor = 'black',  # Color del borde de los puntos
        marker = 'o'       # Forma de los puntos
    )
    
    # Títulos y etiquetas
    ax.set_title('Gráfico de dispersión')
    ax.set_xlabel(x_col)
    ax.set_ylabel(y_col)
    
    # Mostrar el gráfico
    plt.show()

# Suponiendo que df es tu DataFrame que ya ha sido cargado y preprocesado
# plot_data(df, 'Temperature', 'L')  # Ajusta 'Temperature' y 'L' al nombre de tus columnas reales


## **DETERMINACIÓN DE TIPOS DE ESTRELLAS**

 1. Programar k-means, utilizando buenas prácticas de programación, es decir, debe ser una función y comparar los resultados y la eficiencia con la implementación de k-means en scikit-learn

2. Tenemos variables categóricas (Color, Clase Espectral). Tenemos dos
posibilidades: (a) codificar con one-hot-encoding; (b) codificar como una variable
ordinal. Tener en cuenta que el color está asociado a la cantidad de energía, y
algo parecido con la clase espectral.

3. Aplicar al menos dos diferentes algoritmos de clustering de los explicados en clase (K-Means, Hierarchical Clustering/Dendrogramas, DBSCAN), comparando y discutiendo los resultados que se obtienen de ellos

4. Discutir los resultados que se obtienen si a los atributos categóricos se les aplica one-hot-encoding o se asigna un valor numérico a las secuencias (variable ordinal).


5. A partir de los resultados obtenidos, ¿qué pipeline de clustering, es decir, qué transformaciones de datos, algoritmo, con sus hiperparámetros, transformación de datos y análisis de resultados recomendaría realizar?

6. ¿Hay similitudes con los grupos obtenidos en el punto 4? Explicar